In [1]:
#dependencies i love dependencies
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Set up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [5]:
# Visit the Mars Temperature Data site
url = 'https://data-class-mars-challenge.s3.amazonaws.com/Mars/index.html'
browser.visit(url)

In [23]:
#parse it
html = browser.html
html_soup = soup(html, 'html.parser')

In [56]:
table = html_soup.find('table', class_='table')

In [29]:
#get the headers
header_row = table.find('tr').find_all('th')
headers = []
for header in header_row:
    headers.append(header.text)

In [58]:
#get data put it in dataframe
df = pd.DataFrame()
mars_data = {}
rows = table.find_all('tr', class_='data-row')
for row in rows:
    row_data = row.find_all('td')
    for i in range(len(headers)):
        mars_data[headers[i]] = row_data[i].text
    df = df.append(mars_data,ignore_index=True)

In [62]:
#check types
df.dtypes

id                  object
terrestrial_date    object
sol                 object
ls                  object
month               object
min_temp            object
pressure            object
dtype: object

In [69]:
#convert types
df['terrestrial_date']= pd.to_datetime(df['terrestrial_date'])
df['id'] = df['id'].astype('int')
df['sol'] = df['sol'].astype('int')
df['ls'] = df['ls'].astype('int')
df['month'] = df['month'].astype('int')
df['min_temp'] = df['min_temp'].astype('float')
df['pressure'] = df['pressure'].astype('float')

In [71]:
#check types
df.dtypes

id                           int32
terrestrial_date    datetime64[ns]
sol                          int32
ls                           int32
month                        int32
min_temp                   float64
pressure                   float64
dtype: object

In [75]:
print("The amount of months on Mars is " + str(max(df['month'])) + ".")

The amount of months on Mars is 12.


In [77]:
print("The amount of Martian (and not Earth) days worth of data that exist in the scraped dataset is " + str(max(df['sol'])) + ".")

The amount of Martian (and not Earth) days worth of data that exist in the scraped dataset is 1977.


In [78]:
df

,id,terrestrial_date,sol,ls,month,min_temp,pressure
0,2,2012-08-16,10,155,6,-75.0,739.0
1,13,2012-08-17,11,156,6,-76.0,740.0
2,24,2012-08-18,12,156,6,-76.0,741.0
3,35,2012-08-19,13,157,6,-74.0,732.0
4,46,2012-08-20,14,157,6,-74.0,740.0
...,...,...,...,...,...,...,...
1862,1889,2018-02-23,1973,133,5,-78.0,730.0
1863,1892,2018-02-24,1974,134,5,-77.0,729.0
1864,1894,2018-02-25,1975,134,5,-76.0,729.0
1865,1893,2018-02-26,1976,135,5,-77.0,728.0
